In [55]:
import cv2
import matplotlib.pyplot as plt
from matplotlib import image
from matplotlib.patches import Circle, Rectangle
import numpy as np
import os
import re
from scipy import spatial
from scipy import signal
from scipy import ndimage
from scipy import optimize

%matplotlib

Using matplotlib backend: Qt5Agg


## Creating Photogrammetry Targets

In [56]:
def gen_template(w: float, h: float, dpi: int, loc: str, plot=False):
    '''
    Generate an image of a photogrammetry target.

    Parameters
    ----------
    w
        total image width, m
    h
        total image height, m
    dpi
        output image dpi, converts between matrix coordinates and real width
    loc
        [SW, NW, SE, NE, raft] generates a different pattern for each loc
    '''
    np.random.seed(77777) # generate same noise pattern every time

    IN_TO_METERS = 0.0254
    w_in = w / IN_TO_METERS
    h_in = h / IN_TO_METERS
    w_px = np.round(w_in * dpi).astype(int)
    h_px = np.round(h_in * dpi).astype(int)

    fig, ax = plt.subplots()
    fig.set_size_inches(w_in, h_in)
    fig.tight_layout()
    fig.subplots_adjust(left=0., right=1., top=1., bottom=0.)
    ax.scatter([0, 0, w_px, w_px], [0, h_px, 0, h_px], color='k')

    # black background
    # ax.add_patch(Rectangle((0, 0), w_px, h_px, color='k'))
    pattern_size = 8
    pattern = np.random.random((pattern_size,pattern_size)) * 0.5 + .5
    pattern[0][0] = 0
    pattern[pattern_size - 1][pattern_size - 1] = 1
    ax.imshow(ndimage.zoom(pattern, h_px / pattern_size, order=0), cmap='Greys')

    # center cross
    c_size = dpi * 0.01 / IN_TO_METERS # 10 cm
    c_thick = dpi * 0.001 / IN_TO_METERS # 1 mm
    ax.add_patch(Rectangle((w_px / 2 - c_size / 2, h_px / 2 - c_thick / 2), c_size, c_thick, color='w')) # horz
    ax.add_patch(Rectangle((w_px / 2 - c_thick / 2, h_px / 2 - c_size / 2), c_thick, c_size, color='w')) # vert

    circ_diam = c_size
    if loc == 'SW':
        locs = (
            (circ_diam / 2, 0),
            (w_px / 4, h_px / 4),
        )
    elif loc == 'NW':
        locs = (
            (circ_diam / 2, 0),
            (w_px / 4 + circ_diam / 2, 0),
            (w_px / 4, 3 * h_px / 4),
        )
    elif loc == 'NE':
        locs = (
            (circ_diam / 2, 0),
            (w_px / 4 + circ_diam / 2, 0),
            (2 * w_px / 4 + circ_diam / 2, 0),
            (3 * w_px / 4, 3 * h_px / 4),
        )
    elif loc == 'SE':
        locs = (
            (circ_diam / 2, 0),
            (w_px / 4 + circ_diam / 2, 0),
            (2 * w_px / 4 + circ_diam / 2, 0),
            (3 * w_px / 4 + circ_diam / 2, 0),
            (3 * w_px / 4, h_px / 4),
        )
    elif loc == 'raft':
        locs = (
            (w_px / 4, h_px / 4),
            (w_px / 4, 3 * h_px / 4),
            (3 * w_px / 4, 3 * h_px / 4),
            (3 * w_px / 4, h_px / 4),
        )
    else:
        raise ValueError(f'loc {loc} is not an option')

    for elem in locs:
        ax.add_patch(Circle(elem, circ_diam / 2, color='w', zorder=1))

    ax.set_aspect('equal')
    ax.set_xlim([0, w_px])
    ax.set_ylim([0, h_px])
    ax.set_xticks([])
    ax.set_yticks([])
    fig.savefig(f'{loc}_target.png')
    if not plot:
        plt.close()

In [57]:
gen_locs = ['SW', 'NW', 'NE', 'SE', 'raft']
TARGET_W = 0.04 # 40 mm
TARGET_H = TARGET_W
# for loc in locs:
    # gen_template(TARGET_W, TARGET_H, 300, loc, plot=False)